In [77]:
import os
import pathlib
from pathlib import Path 
import pandas as pd
import numpy as np
import re

To do:
* Remove duplicate rows
* Check for different text referring to same facility eg 'hospital' vs 'chp'

## Read in raw data

In [89]:
df_raw = pd.read_excel('Health Facility Assessment (1).xlsx',index_col=0, usecols='B:GW')
df = df_raw.copy()

## Delete empty columns

In [90]:
# the columns whose names end with 'choice_labels' are empty, so we can delete them
choice_label_cols = []
for col in df.columns:
    if len((re.findall('\.choice_labels',col)))!=0:
        choice_label_cols.append(col)

df = df.drop(choice_label_cols,axis=1)


## Shorten column names

Let's start by displaying the column names:

In [91]:
cols = list(df.columns)
print(*cols,sep='\n')

form.health_centre_information.facility_name
form.facility_gps
form.health_centre_information.location_information.region_province
form.health_centre_information.location_information.district
form.health_centre_information.location_information.chiefdom
form.health_centre_information.location_information.facility_location
form.health_centre_information.facility_type
form.health_centre_information.facility_type_other
form.health_centre_information.managing_authority
form.health_centre_information.managing_authority_other
form.health_centre_information.setting
form.health_centre_information.outpatient_only
form.health_centre_information.capacity.number_consultation_rooms
form.health_centre_information.capacity.number_inpatient_beds
form.health_centre_information.capacity.number_maternity_beds
form.health_centre_information.group_number_employed.list_community_health_officer.cho_number_in_post
form.health_centre_information.group_number_employed.list_community_health_officer.cho_number_pre

These column names are painfully long to work with, so I will remove some unnecesary words. 

In [92]:
new_cols = []
for i,name in enumerate(cols):
    name = re.sub('form\.','',name)
    name = re.sub('health_centre_information\.','',name)
    name = re.sub('location_information\.','',name)
    name = re.sub('group_number_employed\.','',name)
    name = re.sub('list_.*\.','',name)
    name = re.sub('ql_information_education_communication\.','',name)
    name = re.sub('ql_human_resources\.','',name)
    name = re.sub('ql_surveillance\.','',name)
    name = re.sub('ql_triage_and_early_recognition\.','',name)
    name = re.sub('ql_chw\.','',name)
    name = re.sub('ql_isolation_physical_distancing\.','',name)
    
    # remove everything from 'grp_infection_prevention_and_control.' to the next '.'
    name = re.sub('grp_infection_prevention_and_control\..*\.','',name) 
    # remove any remaining instances of 'grp_infection_prevention_and_control.'
    name = re.sub('grp_infection_prevention_and_control\.','',name) 
    
    # remove everything before the word 'log'
    name = re.sub('.*\.log_','log_',name)
    
    new_cols.append(name)

    print(name)
df.columns = new_cols

facility_name
facility_gps
region_province
district
chiefdom
facility_location
facility_type
facility_type_other
managing_authority
managing_authority_other
setting
outpatient_only
capacity.number_consultation_rooms
capacity.number_inpatient_beds
capacity.number_maternity_beds
cho_number_in_post
cho_number_present_at_visit
cht_number_in_post
cht_number_present_at_visit
cha_number_in_post
cha_number_present_at_visit
cm_number_in_post
cm_number_present_at_visit
sechn_number_in_post
sechn_number_present_at_visit
lab_tech_number_in_post
lab_tech_number_present_at_visit
mch_aides_number_in_post
mch_aides_number_present_at_visit
cleaner_porter_number_in_post
cleaner_porter_number_present_at_visit
other_number_in_post
other_number_present_at_visit
other_staff_cadre
sections_to_review
consultations.head_count.month_1
consultations.head_count.month_2
consultations.head_count.month_3
consultations.head_count.month_4
consultations.general_outpatient.month_1
consultations.general_outpatient.month_

Abbreviations in the new column names:
* cho: community health officer
* cht: community health technician
* cha: community health assistant
* cm: community midwives
* sechn: state enrolled community health nurse
* mch: maternal and child health
* hr: human resources
* iec: information education communication
* surv: surveillance
* ter: triage and early recognition
* chw: community health worker
* iso: isolation
* ppe: personal protective equipment
* ipc: infection prevention and control
* wcd: waste collection and disposal
* ds: disinfection and sterilization
* log: logistics

## Some preliminary changes to the data

There are a few obvious things we can do right off the bat to make the data easier to work with. 

In [93]:
# replace --- values with NA
df = df.replace('---', np.nan, regex=True)


# make location names all uppercase so that differences in capitalisation don't prevent matches
df['facility_name'] = df['facility_name'].str.upper()
df['region_province'] = df['region_province'].str.upper()
df['district'] = df['district'].str.upper()
df['chiefdom'] = df['chiefdom'].str.upper()
df['facility_location'] = df['facility_location'].str.upper()

# replace 'yes' with True and 'no' with False
df['outpatient_only'] = df['outpatient_only'].apply(lambda x: True if x=='yes' else x)
df['outpatient_only'] = df['outpatient_only'].apply(lambda x: False if x=='no' else x)

# split GPS values into longitude and latitude
df.insert(loc=1,column = 'facility_gps_lat',value=df['facility_gps'].apply(lambda x: x.split(',')[0]))
df.insert(loc=2,column = 'facility_gps_lon',value=df['facility_gps'].apply(lambda x: x.split(',')[1]))
df = df.drop('facility_gps',axis=1)




/tmp/ipykernel_4541/2837350214.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.insert(loc=1,column = 'facility_gps_lat',value=df['facility_gps'].apply(lambda x: x.split(',')[0]))
/tmp/ipykernel_4541/2837350214.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.insert(loc=2,column = 'facility_gps_lon',value=df['facility_gps'].apply(lambda x: x.split(',')[1]))


In [60]:
df

,facility_name,facility_gps_lat,facility_gps_lon,region_province,district,chiefdom,facility_location,facility_type,facility_type_other,managing_authority,...,grp_infection_prevention_and_control.ql_water_sanitation_and_hygiene.score_infection_prevention_and_control_water_sanitation_and_hygiene,grp_infection_prevention_and_control.ql_water_sanitation_and_hygiene.score_max_infection_prevention_and_control_water_sanitation_and_hygiene,grp_infection_prevention_and_control.ql_disinfection_and_sterilization.score_infection_prevention_and_control_disinfection_and_sterilization,grp_infection_prevention_and_control.ql_disinfection_and_sterilization.score_max_infection_prevention_and_control_disinfection_and_sterilization,grp_infection_prevention_and_control.score_infection_prevention_and_control,grp_infection_prevention_and_control.score_max_infection_prevention_and_control,question1.score_logistics_patient_and_sample_transfer,question1.score_max_logistics_patient_and_sample_transfer,score_total,score_max_total
number,,,,,,,,,,,,,,,,,,,,,
0,WILLIAM HOSPITAL,60.234598,-90.157892,SOUTHERN,PUJEHUN,NEW YORK,JASONFORT,community_health_post,NaN,government_public,...,4,5,1,5,10.5,27,19.5,44,48.5,102.0
1,KATHRYN HOSPITAL/CHC,21.2454855,-107.509120,SOUTHERN,BO,ARIZONA,WEST JAMESSHIRE,community_health_post,NaN,government_public,...,1.5,5,0,5,5.5,27,21,44,35.5,102.0
2,JESSICA HOSPITAL,-0.178657,-9.190029,SOUTHERN,BO,NEW MEXICO,ROBERTSSIDE,community_health_post,NaN,government_public,...,2,5,0,5,10,27,33.5,44,55.5,102.0
3,LAUREN CHC,35.856989,-87.512790,SOUTHERN,BONTHE,MISSOURI,PORT CHRISTOPHER,mch_post,NaN,government_public,...,4,5,2,5,12,27,34.5,44,57.5,102.0
4,TAMMY CHC,54.7148835,150.744813,SOUTHERN,BONTHE,VIRGINIA,RENEETON,community_health_post,NaN,government_public,...,4,5,3,5,12,27,29.5,44,55.0,102.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,NICOLE TERTIARY HOSPITAL,63.155728,-82.989076,SOUTHERN,BO,RHODE ISLAND,NEW CHRISTINE,community_health_centre,NaN,government_public,...,2,5,0,5,11.5,27,18.5,44,45.5,102.0
141,NICOLE HOSPITAL,-56.2934725,-143.818347,SOUTHERN,BO,VIRGINIA,STEVECHESTER,community_health_post,NaN,government_public,...,4,5,1.5,5,9.5,27,32,44,46.0,102.0
142,LATOYA CHC,-18.108589,157.038699,NORTHERN,TONKOLILI,MASSACHUSETTS,EAST MARCUS,community_health_centre,NaN,government_public,...,1.5,5,1.5,5,10,27,16.5,44,50.0,102.0


Let's investigate duplicate facility names:

In [61]:
#df[df.duplicated(['facility_name','facility_location'],keep=False)].sort_values('facility_location').to_csv('temp.csv')

In [62]:
df[df.duplicated(['facility_name','facility_location'],keep=False)].sort_values('facility_location')

,facility_name,facility_gps_lat,facility_gps_lon,region_province,district,chiefdom,facility_location,facility_type,facility_type_other,managing_authority,...,grp_infection_prevention_and_control.ql_water_sanitation_and_hygiene.score_infection_prevention_and_control_water_sanitation_and_hygiene,grp_infection_prevention_and_control.ql_water_sanitation_and_hygiene.score_max_infection_prevention_and_control_water_sanitation_and_hygiene,grp_infection_prevention_and_control.ql_disinfection_and_sterilization.score_infection_prevention_and_control_disinfection_and_sterilization,grp_infection_prevention_and_control.ql_disinfection_and_sterilization.score_max_infection_prevention_and_control_disinfection_and_sterilization,grp_infection_prevention_and_control.score_infection_prevention_and_control,grp_infection_prevention_and_control.score_max_infection_prevention_and_control,question1.score_logistics_patient_and_sample_transfer,question1.score_max_logistics_patient_and_sample_transfer,score_total,score_max_total
number,,,,,,,,,,,,,,,,,,,,,
47,JOANNA CLINIC,60.1909345,162.787117,SOUTHERN,BO,MISSISSIPPI,AMBERCHESTER,mch_post,NaN,government_public,...,0.5,5,0.5,5,3.5,27,11,44,22.5,102.0
25,JOANNA CLINIC,15.7540145,-153.871776,SOUTHERN,BO,MISSISSIPPI,AMBERCHESTER,mch_post,NaN,government_public,...,0.5,5,0.5,5,3.5,27,12,44,22.0,102.0
86,MELISSA CLINIC,35.576733,179.184687,NORTHERN,KOINADUGU,TENNESSEE,ANDREACHESTER,community_health_post,NaN,government_public,...,0.5,5,3,5,13.5,27,16,44,55.0,102.0
113,MELISSA CLINIC,-29.0475445,-115.874086,NORTHERN,KOINADUGU,TENNESSEE,ANDREACHESTER,community_health_post,NaN,government_public,...,1,5,3,5,13,27,14.5,44,52.5,102.0
96,RICHARD MCHP,-26.3419165,-172.661039,WESTERN,WESTERN AREA URBAN,KENTUCKY,COLLEENBURGH,community_health_post,NaN,other,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
97,RICHARD MCHP,-81.3326275,69.862048,WESTERN,WESTERN AREA URBAN,KENTUCKY,COLLEENBURGH,community_health_post,NaN,other,...,3,5,3.5,5,15.5,27,8,44,26.5,102.0
48,AMY CHP,84.498725,48.271191,WESTERN,WESTERN AREA URBAN,WISCONSIN,EAST JAMESBERG,community_health_post,NaN,government_public,...,NaN,NaN,NaN,NaN,NaN,NaN,8.5,44,8.5,44.0
87,AMY CHP,70.462361,122.083912,WESTERN,WESTERN AREA URBAN,WISCONSIN,EAST JAMESBERG,community_health_post,NaN,government_public,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,4.0
95,JOHN CLINIC,53.647814,-115.559792,NORTHERN,KOINADUGU,TENNESSEE,GARYTOWN,mch_post,NaN,government_public,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0


## Data types

Let us explore the data types:

In [63]:
df.dtypes

facility_name                                                                       object
facility_gps_lat                                                                    object
facility_gps_lon                                                                    object
region_province                                                                     object
district                                                                            object
                                                                                    ...   
grp_infection_prevention_and_control.score_max_infection_prevention_and_control     object
question1.score_logistics_patient_and_sample_transfer                               object
question1.score_max_logistics_patient_and_sample_transfer                           object
score_total                                                                        float64
score_max_total                                                                    float64

There are lots of 'object' data types. Let's make the dataframe easier to use for modelling by assigning the correct data types. There are a lot of columns, so I assign data types in a bit of a hacky way below to save time. I consider the part of the column name after the last full stop. If it contains a particular keyword, I give it a particular data type. I use 16 bits for the integers because it's very unlikely that any facility will have more than around 32000 of any of the quantities in this database. 

In [70]:
type_dict = {} # make a dictionary specifying data types for columns 
for col in df.columns:
    name_parts = col.split('.')
    
    if len(re.findall('number_', name_parts[-1]))!=0:
        type_dict[col] = 'Int16'
    if name_parts[-1][:6]=='month_':
        type_dict[col] = 'Int16'
    elif name_parts[-1][:3]=='hr_':
        type_dict[col] = 'string'
    elif name_parts[-1][:4]=='iec_':
        type_dict[col] = 'string'
    elif name_parts[-1][:5]=='surv_':
        type_dict[col] = 'string'
    elif name_parts[-1][:4]=='ter_':
        type_dict[col] = 'string'
    elif name_parts[-1][:4]=='chw_':
        type_dict[col] = 'string'
    elif name_parts[-1][:4]=='iso_':
        type_dict[col] = 'string'
    elif name_parts[-1][:4]=='ipc_':
        type_dict[col] = 'string'
    elif name_parts[-1][:4]=='log_':
        type_dict[col] = 'string'
    elif name_parts[-1][:15]=='monthly_average':
        type_dict[col] = 'float64'
    elif name_parts[-1][:6]=='score_':
        type_dict[col] = 'float64'
type_dict_2 = {'facility_name': 'string', 'facility_gps_lat': 'float64', 'facility_gps_lon':'float64', 'region_province': 'string', 'district': 'string', 'chiefdom': 'string', 'facility_location': 'string', 'facility_type': 'string', 'facility_type_other': 'string', 'managing_authority': 'string', 'managing_authority_other': 'string', 'setting': 'string', 'outpatient_only': 'boolean' }
type_dict = type_dict | type_dict_2 # join two dictionaries

df=df.astype(type_dict)

In [71]:
df.dtypes

facility_name                                                                       string
facility_gps_lat                                                                   float64
facility_gps_lon                                                                   float64
region_province                                                                     string
district                                                                            string
                                                                                    ...   
grp_infection_prevention_and_control.score_max_infection_prevention_and_control    float64
question1.score_logistics_patient_and_sample_transfer                              float64
question1.score_max_logistics_patient_and_sample_transfer                          float64
score_total                                                                        float64
score_max_total                                                                    float64

In [72]:
[i for i in df.columns if df[i].dtype=='Int16']

['capacity.number_consultation_rooms',
 'capacity.number_inpatient_beds',
 'capacity.number_maternity_beds',
 'cho_number_in_post',
 'cho_number_present_at_visit',
 'cht_number_in_post',
 'cht_number_present_at_visit',
 'cha_number_in_post',
 'cha_number_present_at_visit',
 'cm_number_in_post',
 'cm_number_present_at_visit',
 'sechn_number_in_post',
 'sechn_number_present_at_visit',
 'lab_tech_number_in_post',
 'lab_tech_number_present_at_visit',
 'mch_aides_number_in_post',
 'mch_aides_number_present_at_visit',
 'cleaner_porter_number_in_post',
 'cleaner_porter_number_present_at_visit',
 'other_number_in_post',
 'other_number_present_at_visit',
 'consultations.head_count.month_1',
 'consultations.head_count.month_2',
 'consultations.head_count.month_3',
 'consultations.head_count.month_4',
 'consultations.general_outpatient.month_1',
 'consultations.general_outpatient.month_2',
 'consultations.general_outpatient.month_3',
 'consultations.general_outpatient.month_4',
 'consultations.de

In [256]:
print(df['form.health_centre_information.location_information.facility_location'].unique())

KeyError: 'form.health_centre_information.location_information.facility_location'

In [93]:
df.to_csv('health_facility_assessment_cleaned.csv', header=True, index=True, index_label=None)